### A) Pre-requisites - Importing Packages and Functions

In [1]:
import pandas as pd
import numpy as np
from io import StringIO

#For text processing

import re
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# For Vectorization

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
# For modeling

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score

from sklearn_pandas import DataFrameMapper

import matplotlib.pyplot as plt

### B) Reading Input Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv("/content/drive/My Drive/DATA/AI Export.csv",sep=",", encoding='cp1252')

### C) Exploratory Data Analysis (EDA)

1. Columns Description

In [10]:
print(df.columns)
print(df.shape)

Index(['AI_DATE', 'AC_MODEL', 'AC_SN', 'OPERATOR_CODE',
       'EVENT_RECORD_IDENTIFIER', 'DIVERTED_LANDING', 'AIR_TURN_BACK',
       'REJECTED_TAKE_OFF', 'GROUND_TURN_BACK', 'ABORTED_APPROACH',
       'EMERGENCY_DESCENT', 'SMOKE', 'EMERGENCY_LANDING', 'INFLIGHT_SHUTDOWN',
       'GEN_AIR_INTERRUPT', 'GEN_GROUND_INTERRUPT', 'ATA_SYMPTOM', 'ATA_CAUSE',
       'INTERRUPTION_REASON', 'CORRECTIVE_ACTION', 'AI_ID', 'EDIT_DATE',
       'EDITED_BY'],
      dtype='object')
(51781, 23)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51781 entries, 0 to 51780
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AI_DATE                  51781 non-null  object 
 1   AC_MODEL                 51781 non-null  object 
 2   AC_SN                    51781 non-null  int64  
 3   OPERATOR_CODE            51781 non-null  object 
 4   EVENT_RECORD_IDENTIFIER  10541 non-null  object 
 5   DIVERTED_LANDING         51781 non-null  object 
 6   AIR_TURN_BACK            51781 non-null  object 
 7   REJECTED_TAKE_OFF        51781 non-null  object 
 8   GROUND_TURN_BACK         51781 non-null  object 
 9   ABORTED_APPROACH         51781 non-null  object 
 10  EMERGENCY_DESCENT        51781 non-null  object 
 11  SMOKE                    51781 non-null  object 
 12  EMERGENCY_LANDING        51781 non-null  object 
 13  INFLIGHT_SHUTDOWN        51781 non-null  object 
 14  GEN_AIR_INTERRUPT     

Variables with Null Values: 

In [ ]:
df[df.columns[df.isna().any()]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51781 entries, 0 to 51780
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EVENT_RECORD_IDENTIFIER  10541 non-null  object 
 1   ATA_SYMPTOM              22962 non-null  float64
 2   CORRECTIVE_ACTION        43943 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.2+ MB


In [ ]:
df.head()

2. Numerical data

In [ ]:
df.describe()

3. Target Variable

In [ ]:
df.ATA_SYMPTOM

4. Slicing Data Frame to Only Include the Target and Predictor Variables

In [12]:
df_filter = df[['AC_MODEL',"ATA_SYMPTOM","ATA_CAUSE","INTERRUPTION_REASON","CORRECTIVE_ACTION"]]
df_filter.shape

(51781, 5)

5. Removing Null Values of Target Variable (don't do this step por ATA CAUSE because it has no null values)

In [13]:
df_filter = df_filter.dropna(axis=0,subset=['ATA_SYMPTOM'])
df_filter.shape

(22962, 5)

### D) Data Wrangling

#### 1. Converting target variable from float to int

In [14]:
df_transformed = df_filter
df_transformed.ATA_SYMPTOM = df_transformed.ATA_SYMPTOM.astype("int64")
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22962 entries, 0 to 51780
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AC_MODEL             22962 non-null  object
 1   ATA_SYMPTOM          22962 non-null  int64 
 2   ATA_CAUSE            22962 non-null  int64 
 3   INTERRUPTION_REASON  22962 non-null  object
 4   CORRECTIVE_ACTION    18325 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


#### 2. Imputing an empty string to the CORRECTIVE_ACTION null values

In [15]:
df_transformed.CORRECTIVE_ACTION = df_transformed.CORRECTIVE_ACTION.fillna('')
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22962 entries, 0 to 51780
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AC_MODEL             22962 non-null  object
 1   ATA_SYMPTOM          22962 non-null  int64 
 2   ATA_CAUSE            22962 non-null  int64 
 3   INTERRUPTION_REASON  22962 non-null  object
 4   CORRECTIVE_ACTION    22962 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


#### 3. Capturing the ATA_COMPONENTS, ATA_SECTION and ATA_CHAPTER information from ATA_SYMPTOM and ATA_CAUSE

In [ ]:
#ATA_SYMPTOM is a numeric field

#Description of one-digit ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM < 10].ATA_SYMPTOM.unique()

#Description of two-digits ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM.between(10, 99)].ATA_SYMPTOM.unique()

#Description of three-digits ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM.between(100, 999)].ATA_SYMPTOM.unique()

#Description of four-digits ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM.between(1000, 9999)].ATA_SYMPTOM.unique()

#Description of five-digits ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM.between(10000, 99999)].ATA_SYMPTOM.unique()

#Description of six-digits ATA_SYMPTOMs
df_transformed[df_transformed.ATA_SYMPTOM.between(100000, 999999)].ATA_SYMPTOM.unique()

#Description of invalid ATA_SYMPTOMs (more than six digits)
df_transformed[df_transformed.ATA_SYMPTOM > 999999].ATA_SYMPTOM.unique()

In [16]:
#Function to convert the ATA_number to text and assign the value to the correct variable
# ATA_Numbers with two-digits are ATA Chapters
# ATA_Numbers with four-digits are ATA Sections
# ATA_Numbers with two-digits are ATA Components

def ATA_number_to_text(ATA_Number):
    ATA_Chapter = None
    ATA_Section = None
    ATA_Components = None
    if ATA_Number < 10: #one-digit
        ATA_Chapter = '0' + str(ATA_Number)
    elif ATA_Number < 100: #two-digits
        ATA_Chapter = str(ATA_Number)
    elif ATA_Number < 1000: #three-digits
        ATA_Chapter = str(ATA_Number)[0:2]
        ATA_Section = '0' + str(ATA_Number)
    elif ATA_Number < 10000: #four-digits
        ATA_Chapter = str(ATA_Number)[0:2]
        ATA_Section = str(ATA_Number)
    elif ATA_Number < 100000: #five-digits
        print("Invalid ATA_Number") #All ATA_Components start with 1
    elif ATA_Number < 1000000: #six-digits
        ATA_Chapter = str(ATA_Number)[0:2]
        ATA_Section = str(ATA_Number)[0:4]     
        ATA_Components = str(ATA_Number)
    else:
        print("Invalid ATA_Number")
    return ATA_Chapter, ATA_Section, ATA_Components

In [ ]:
#Function testing
ATA_number_to_text(1200)

In [ ]:
df_transformed.tail()

In [17]:
df_transformed[['ATA_SYMP_CH', 'ATA_SYMP_SEC','ATA_SYMP_COMP']] = pd.DataFrame(df_transformed.ATA_SYMPTOM.apply(ATA_number_to_text).tolist(), index=df_transformed.index)
df_transformed[['ATA_CAUSE_CH', 'ATA_CAUSE_SEC','ATA_CAUSE_COMP']] = pd.DataFrame(df_transformed.ATA_CAUSE.apply(ATA_number_to_text).tolist(), index=df_transformed.index)

#### 4. Converting AC_MODEL to a Dummy Variable

In [18]:
df_transformed["AC_MODEL_CRJ"] = pd.Series(dtype='int64') #Initializing column with zeros/False
df_transformed.loc[df['AC_MODEL'] == 'CRJ', 'AC_MODEL_CRJ'] = 1 #Assigning 1 (True) to AC_MODEL_CRJ when 'CRJ'
df_transformed.loc[df['AC_MODEL'] == 'CRJ700', 'AC_MODEL_CRJ'] = 0 #Assigning 0 (False) to AC_MODEL_CRJ when 'CRJ700'

In [19]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22962 entries, 0 to 51780
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AC_MODEL             22962 non-null  object 
 1   ATA_SYMPTOM          22962 non-null  int64  
 2   ATA_CAUSE            22962 non-null  int64  
 3   INTERRUPTION_REASON  22962 non-null  object 
 4   CORRECTIVE_ACTION    22962 non-null  object 
 5   ATA_SYMP_CH          22962 non-null  object 
 6   ATA_SYMP_SEC         22298 non-null  object 
 7   ATA_SYMP_COMP        1389 non-null   object 
 8   ATA_CAUSE_CH         22962 non-null  object 
 9   ATA_CAUSE_SEC        22298 non-null  object 
 10  ATA_CAUSE_COMP       9212 non-null   object 
 11  AC_MODEL_CRJ         22962 non-null  float64
dtypes: float64(1), int64(2), object(9)
memory usage: 2.3+ MB


### E) Text Preprocessing 

#### 1. Stemming (Try Lemmatization later)

In [20]:
stemmer = PorterStemmer()
def identify_tokens_stem(df_column):
    #df_column = re.sub(r'\d',' ',df_column)
    words = nltk.word_tokenize(df_column)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment
df_transformed['REASON_STEM'] = df_transformed["INTERRUPTION_REASON"].apply(identify_tokens_stem)
df_transformed['ACTION_STEM'] = df_transformed["CORRECTIVE_ACTION"].apply(identify_tokens_stem)

In [ ]:
df_transformed.info()

In [ ]:
df_transformed.head()

#### 2. Vectorization

In [ ]:
vectorizer = TfidfVectorizer(max_features=40000,stop_words='english').fit(df_transformed['REASON_STEM'])

### F) Modeling (Indiano - Versão Inicial with SVM)

#### 1. Train and Test Split

In [21]:
#Predictor Variables
X = df_transformed['REASON_STEM']
X = df_transformed['REASON_STEM']+df_transformed['ACTION_STEM']
#X = df_transformed[['REASON_STEM','ACTION_STEM']]

#Target Variable
y = df_transformed['ATA_CAUSE']

#80-20 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train

42707         nose gear door disagre master warn after t/o
50445    bird strike capt windshieldclean bug gut and i...
47414    possibl bird strike left wing lead edgperform ...
45923    ***** pie event *****wing overheat warn ON tak...
35603              wing a/i fault - staturesecur connector
                               ...                        
40464                                reason : dummi record
50394    FD 1 fail statu messagreset system and fault c...
31888    stall fail caution msg after take off [ a/c re...
3663                                 reason : dummi record
44601    incid : rtrn deic truck advis the feeler wand ...
Length: 18369, dtype: object

#### 2. Classification Algorithm - SVM (Support Vector Machine)

In [23]:
#clf = make_pipeline(LinearSVC())
#clf.fit(X_train, y_train)

#pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=40000,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=40000,stop_words='english')),('clf',SVC())])


#### 3. Create and Test Model

In [24]:
pipe.fit(X_train,y_train)
predicted = pipe.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

0.5752231656869149


### G. Modelling (Will) 

#### Original Configuration (Interruption Reason and Corrective Actions text fields are concatenated)

In [ ]:
# Original Configuration (Reason and Action are concatenated)
# Target Variable: ATA_CAUSE
# No distinction between airplane models (CRJ vs CRJ700)

X = df_transformed['REASON_STEM'] + df_transformed['ACTION_STEM']
y = df_transformed['ATA_CAUSE']

In [ ]:
len(y.unique())

705

In [ ]:
# Maximum 2000 features to decrease processing time
vectorizer = TfidfVectorizer(max_features=4000,stop_words='english')
features = vectorizer.fit_transform(X)

In [ ]:
features.shape

(13454, 4000)

In [ ]:
# 80-20 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, y, 
                                                    test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(10763, 4000)

In [ ]:
X_test.shape

(2691, 4000)

In [ ]:
#Modeling with Classifier
#clf = SVC()
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = clf.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

0.5671674286958415


In [ ]:
def printreport(exp, pred):
    print(pd.crosstab(exp, pred, rownames=['Actual'], colnames=['Predicted']))
 
    print('\n \n')
    print(classification_report(exp, pred))
 
printreport(y_test, predicted)

Predicted  0       200     240     550     2124    2130    2131    2151    \
Actual                                                                      
0             113       1       0       0       0       0       0       0   
200             0      33      10       2       0       0       1       1   
240             0       2     427       1       0       0       0       2   
320             0       1       0       0       0       0       0       0   
520             0       1       2       0       0       0       0       0   
...           ...     ...     ...     ...     ...     ...     ...     ...   
801004          0       0       0       0       0       0       0       0   
801010          0       0       0       0       0       0       0       0   
801101          0       0       1       0       0       0       0       0   
801103          0       0       0       0       0       0       0       0   
801105          0       0       0       0       0       0       0       0   

C:\Users\willi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\willi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\willi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\willi\anaconda3\lib\site-packages\sklearn\metrics\_cla

#### Original Configuration (Interruption Reason and Corrective Actions text fields are concatenated) + Differentiating Aircrafts

In [ ]:
# Original Configuration (Reason and Action are concatenated)
# Target Variable: ATA_CAUSE
# Distinct airplane models (CRJ vs CRJ700)

X = df_transformed[df_transformed['AC_MODEL'] != 'CRJ']
y = X['ATA_CAUSE']
X = X['REASON_STEM'] + X['ACTION_STEM']

In [ ]:
# Maximum 2000 features to decrease processing time
vectorizer = TfidfVectorizer(max_features=4000,stop_words='english')
features = vectorizer.fit_transform(X)

In [ ]:
# 80-20 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, y, 
                                                    test_size=0.2, random_state=42)

In [ ]:
#Modeling with Classifier
#clf = SVC()
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = clf.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

0.6172426607209216


#### Adding other existing features to the model

a. Processing Text as Different Features (Interruption Reasons and Corrective Actions have distinct meanings)

In [ ]:
# Reason and Action are no concatenated anymore
# Target Variable: ATA_CAUSE
# Still no distinction between airplane models (CRJ vs CRJ700)

X = df_transformed[['REASON_STEM','ACTION_STEM']]
y = df_transformed['ATA_CAUSE']

In [ ]:
#Combine the features of the transformed column
#Keep 2000 features per text as a maximum to avoid longer processing times
mapper = DataFrameMapper([
     ('REASON_STEM', TfidfVectorizer(max_features=2000,stop_words='english')),
     ('ACTION_STEM', TfidfVectorizer(max_features=2000,stop_words='english'))
 ])

features = mapper.fit_transform(X)
categories = y

In [ ]:
features.shape

(22962, 4000)

In [ ]:
# 80-20 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, y, 
                                                    test_size=0.2, random_state=42)

In [ ]:
#Modeling with Classifier
#clf = SVC()
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = clf.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

0.6037448290877422


b. Splitting Data by Aircract Model

In [ ]:
# Reason and Action are no concatenated anymore
# Aircraft Model as an additional feature
# Target Variable: ATA_CAUSE

X = df_transformed[df_transformed['AC_MODEL'] != 'CRJ'][['REASON_STEM','ACTION_STEM']]
y = df_transformed[df_transformed['AC_MODEL'] != 'CRJ']['ATA_CAUSE']

In [ ]:
y.shape

(13454,)

In [ ]:
#Combine the features of the transformed column
#Keep 2000 features per text as a maximum to avoid longer processing times
mapper = DataFrameMapper([
     ('REASON_STEM', TfidfVectorizer(max_features=2000,stop_words='english')),
     ('ACTION_STEM', TfidfVectorizer(max_features=2000,stop_words='english')),
 ])

features = mapper.fit_transform(X)
categories = y

In [ ]:
features.shape

(13454, 4000)

In [ ]:
# 80-20 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, y, 
                                                    test_size=0.2, random_state=42)

In [ ]:
#Modeling with Classifier
#clf = SVC()
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = clf.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

0.6610925306577481


In [ ]:
#Classifier   #Target Var    #Features               #Test Size    #Accuracy
#LogisticReg  ATA_CAUSE      Text Concat (2k feat)   20%           57.06%
#LogisticReg  ATA_CAUSE      Text Concat (4k feat)   20%           56,71%
#LogisticReg  ATA_CAUSE      Text Concat+Model       20%           54,83% (CRJ) e 63,02% (CRJ700)
#LogisticReg  ATA_CAUSE      Text Concat+Model(4k)   20%           54,25% (CRJ) e 61,72% (CRJ700)
#LogisticReg  ATA_CAUSE      Reason+Action(4k)       20%           60.06%
#LogisticReg  ATA_CAUSE      Reason+Action(4k)+Model 20%           58,25% (CRJ) e 66,10% (CRJ700)